# Experiment

In mlflow, an experiment is a structure that contains a set of runs. Typically, comparable runs are kept in the same experiment.

In [2]:
import mlflow
mlflow.set_tracking_uri("file:///tmp/experiment")

## Experiment object

There is an object that describes an experiment in mlflow. It has the type `mlflow.entities.experiment.Experiment`.

You can access the experiment objects in several cases:

- `mlflow.set_experiment`: returns the experiment that is active now.
- `mlflow.get_experiment`: returns the experiment with the specified id.
- `mlflow.get_experiment_by_name`: returns the experiment with the specified name.

---

The following cell shows the attributes of an experiment.

In [ ]:
experiment = mlflow.set_experiment("some experiment")
print(
    experiment.artifact_location,
    experiment.creation_time,
    experiment.experiment_id,
    experiment.last_update_time,
    experiment.lifecycle_stage,
    experiment.name,
    sep="\n"
)

mlflow-artifacts:/316075063118674621
1754732470778
316075063118674621
1754732470778
active
some experiment


## Current experiment

There is no special tool to get information about the experiment that is active in mlflow right now. The only option that is available now is to start a run and load information from it, using the `run.info.experiment_id` attributes to get the id of the experiment and the `mlflow.get_experiment` function to load the other information about the experiment.

---

The following cell shows how information about the currently used experiment can be loaded using the `experiment_id` that the run keeps.

In [ ]:
with mlflow.start_run() as run:
    experiment_id = run.info.experiment_id
    print("Experiment ID:", experiment_id)
    experiment = mlflow.get_experiment(experiment_id)
    print("Experiment name:", experiment.name)

Experiment ID: 0
Experiment name: Default
🏃 View run secretive-horse-172 at: http://localhost:5000/#/experiments/0/runs/85fd8830eb3a435fbc84142f9083719c
🧪 View experiment at: http://localhost:5000/#/experiments/0


## Env variable

You can set the experiment to be used with `MLFLOW_EXPERIMENT_NAME` environment variable.

---

The following code sets the value `"env experiment"` for the `MLFLOW_EXPERIMENT_NAME` variable.

In [ ]:
import os
import mlflow
os.environ["MLFLOW_EXPERIMENT_NAME"] = "env experiment"

The next code, by loading information about the active run, shows that the value from the environment variable was applied.

In [ ]:
with mlflow.start_run() as run:
    name = mlflow.get_experiment(run.info.experiment_id).name
print(name)

env experiment


## Delete

Use the method `mlflow.delete_experiment` to delete the mlflow experiment. After invoking the method, the experiment will be moved to the `trash`, and you will lose access to it from all MLFlow clients.

You are also restricted from creating a new experiment with the same name. The only way to completely remove an experiment is to delete it from MLFlow Storage.

---

The following cell will create an experiment that we will attempt to delete.

In [11]:
mlflow.create_experiment("delete me")
delete_id = mlflow.get_experiment_by_name("delete me").experiment_id
delete_id

'692879448156854806'

Pay attention to the experiment id.

Calling `mlflow.delete_experiment` goes smothly.

In [12]:
mlflow.delete_experiment(delete_id)

The attempt to recreate the experiment fails.

In [13]:
try:
    mlflow.create_experiment("delete me")
except Exception as e:
    print(e)

Experiment 'delete me' already exists in deleted state. You can restore the experiment, or permanently delete the experiment from the .trash folder (under tracking server's root folder) in order to use this experiment name again.


As `delete me` already exists but in **deleted state**.

Deleted experiments are stored in a special place of the storage. In case of the local file system storage, it is the `.trash` folder.

In [ ]:
!ls /tmp/experiment/.trash

692879448156854806


**Note** that the name of the folder in the `.trash` folder corresponds to the ID of the deleted experiment.

In [19]:
!rm -r /tmp/experiment/.trash/*

After removing the experiment from the `.trash` folder, creating another experiment with the same name does not cause any issues.

In [20]:
mlflow.create_experiment("delete me")

'673998879641046421'